### Neural Network

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split


In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from math import sqrt

##### Load the SQL db for later comparison

In [3]:
import sqlite3
conn=sqlite3.connect("parametres.db")
cur=conn.cursor()

##### Load database sampled to a 25%

In [4]:
df=pd.read_csv('delayed25.csv', index_col=False, warn_bad_lines=True, error_bad_lines=False)
# as the process take too much time, we resample the database 
df = df.sample(frac =.1)
df=df.drop(['Unnamed: 0'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48419 entries, 158705 to 475174
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ActualElapsedTime  48419 non-null  float64
 1   AirTime            48419 non-null  float64
 2   ArrDelay           48419 non-null  int64  
 3   DepDelay           48419 non-null  float64
 4   Distance           48419 non-null  float64
 5   TaxiIn             48419 non-null  float64
 6   TaxiOut            48419 non-null  float64
 7   Cancelled          48419 non-null  float64
 8   Diverted           48419 non-null  int64  
 9   CarrierDelay       48419 non-null  float64
 10  WeatherDelay       48419 non-null  float64
 11  NASDelay           48419 non-null  float64
 12  SecurityDelay      48419 non-null  float64
 13  LateAircraftDelay  48419 non-null  float64
 14  DepTime_sin        48419 non-null  float64
 15  DepTime_cos        48419 non-null  float64
 16  Month_sin       

##### Split attibutes and target

In [5]:
target_column = ['ArrDelay'] 
predictors = list(set(list(df.columns))-set(target_column))
df[predictors] = df[predictors]/df[predictors].max()

df.describe()

,ActualElapsedTime,AirTime,ArrDelay,DepDelay,Distance,TaxiIn,TaxiOut,Cancelled,Diverted,CarrierDelay,...,LateAircraftDelay,DepTime_sin,DepTime_cos,Month_sin,Month_cos,DayOfWeek_sin,DayOfWeek_cos,origin_freq,dest,carrier_me
count,48419.000000,48419.000000,48419.000000,48419.000000,48419.000000,48419.000000,48419.000000,48419.000000,48419.000000,48419.000000,...,48419.000000,48419.000000,48419.000000,4.841900e+04,4.841900e+04,48419.000000,48419.000000,48419.000000,48419.000000,48419.000000
mean,0.499586,0.500283,41.853880,0.016447,0.036253,0.502837,0.497787,0.000310,0.003697,0.012662,...,0.014252,-0.396306,-0.277026,1.053203e-01,2.822428e-02,0.002986,-0.011282,0.340081,0.336523,0.762627
std,0.288631,0.288050,54.697786,0.044838,0.131367,0.287209,0.288041,0.017598,0.060690,0.035075,...,0.031210,0.612072,0.625765,6.596759e-01,7.436123e-01,0.706132,0.725226,0.307294,0.291003,0.138885
min,0.000240,0.000188,-53.000000,-0.015817,-0.132231,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-1.000000,-1.000000,-1.000000e+00,-1.000000e+00,-1.000000,-0.900969,0.000076,0.000110,0.384455
25%,0.247247,0.249259,9.000000,-0.010545,-0.061065,0.250257,0.222222,0.000000,0.000000,0.000000,...,0.000000,-0.918791,-0.856718,-5.000000e-01,-8.660254e-01,-0.801938,-0.900969,0.104784,0.111787,0.659196
50%,0.500000,0.498518,25.000000,0.000000,0.000000,0.559574,0.475976,0.000000,0.000000,0.000000,...,0.000000,-0.623197,-0.425779,1.224647e-16,6.123234e-17,0.000000,-0.222521,0.235995,0.258176,0.819271
75%,0.751251,0.749279,56.000000,0.025483,0.089761,0.749269,0.752252,0.000000,0.000000,0.009564,...,0.015748,-0.010472,0.278991,8.660254e-01,8.660254e-01,0.801938,0.623490,0.431925,0.448304,0.860134
max,1.000000,1.000000,1143.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000


##### Train i test sets

In [6]:
X = df[predictors].values
y = df[target_column].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
print(X_train.shape); print(X_test.shape)

(33893, 22)
(14526, 22)


In [7]:
# Define model previously tested with Kaggle

model = Sequential()
model.add(Dense(100, input_dim=22, activation='relu'))
# check to see if the regression node should be added
model.add(Dense(1))

model.summary() #Print model Summary

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               2300      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 2,401
Trainable params: 2,401
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])

model.fit(X_train, y_train, epochs=200)

Epoch 1/200
1060/1060 [==============================] - 9s 800us/step - loss: 4125.5968 - mean_squared_error: 4125.5968
Epoch 2/200
1060/1060 [==============================] - 1s 803us/step - loss: 2671.3494 - mean_squared_error: 2671.3494
Epoch 3/200
1060/1060 [==============================] - 1s 832us/step - loss: 1665.2918 - mean_squared_error: 1665.2918
Epoch 4/200
1060/1060 [==============================] - 1s 843us/step - loss: 797.0644 - mean_squared_error: 797.0644
Epoch 5/200
1060/1060 [==============================] - 1s 855us/step - loss: 284.3680 - mean_squared_error: 284.3680
Epoch 6/200
1060/1060 [==============================] - 1s 846us/step - loss: 138.8817 - mean_squared_error: 138.8817
Epoch 7/200
1060/1060 [==============================] - 1s 858us/step - loss: 103.0801 - mean_squared_error: 103.0801
Epoch 8/200
1060/1060 [==============================] - 1s 874us/step - loss: 93.3496 - mean_squared_error: 93.3496
Epoch 9/200
1060/1060 [=====================

1060/1060 [==============================] - 1s 1ms/step - loss: 24.3990 - mean_squared_error: 24.3990
Epoch 141/200
1060/1060 [==============================] - 1s 1ms/step - loss: 24.8394 - mean_squared_error: 24.8394
Epoch 142/200
1060/1060 [==============================] - 1s 1ms/step - loss: 23.2443 - mean_squared_error: 23.2443
Epoch 143/200
1060/1060 [==============================] - 1s 1ms/step - loss: 24.5269 - mean_squared_error: 24.5269
Epoch 144/200
1060/1060 [==============================] - 1s 1ms/step - loss: 23.9218 - mean_squared_error: 23.9218
Epoch 145/200
1060/1060 [==============================] - 1s 1ms/step - loss: 24.1770 - mean_squared_error: 24.1770
Epoch 146/200
1060/1060 [==============================] - 1s 1ms/step - loss: 24.1439 - mean_squared_error: 24.1439
Epoch 147/200
1060/1060 [==============================] - 1s 1ms/step - loss: 23.8508 - mean_squared_error: 23.8508
Epoch 148/200
1060/1060 [==============================] - 1s 1ms/step - loss:

In [8]:
pred_train= model.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train)))

pred= model.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred))) 

69.44142954173822
67.66494205823776


In [9]:
from sklearn.metrics import mean_absolute_error
y_pred = model.predict(X_test)

mae=mean_absolute_error(y_test, y_pred)
print('MAE: %.4f' % mae)

mse = mean_squared_error(y_test, y_pred)
print('MSE: %.3f' % mse)
print('RMSE: %.3f' % (mse*(1/2.0)))

from sklearn.metrics import r2_score
y_pred = model.predict(X_test)
r2 = r2_score(y_test,y_pred)
print('R-Squared: %.4f' % r2)


MAE: 42.9658
MSE: 4578.544
RMSE: 2289.272
R-Squared: -0.6152


In [10]:
#SQL database update
code='nn1'
process='Neural_network'
datab='standard'
depdelay='yes'

cur.execute("INSERT OR REPLACE INTO parametres(codi,proces,DepDelay, base_dades, MAE, R2, MSE) VALUES(?,?,?,?,?,?,?)",(code,process,depdelay,datab,'%.3f' % mae,'%.3f' % r2,'%.3f' % mse))
conn.commit()

##### Delete DepDelay

In [11]:
dfdel=df.drop(['DepDelay'], axis=1)

In [12]:
#separem atributs del target

target_column = ['ArrDelay'] 
predictors = list(set(list(dfdel.columns))-set(target_column))
dfdel[predictors] = dfdel[predictors]/dfdel[predictors].max()

In [13]:
X = dfdel[predictors].values
y = dfdel[target_column].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
print(X_train.shape); print(X_test.shape)

(33893, 21)
(14526, 21)


In [14]:
# Define model

model = Sequential()
model.add(Dense(100, input_dim=21, activation='relu'))
# check to see if the regression node should be added
model.add(Dense(1))

model.summary() #Print model Summary

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 100)               2200      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 2,301
Trainable params: 2,301
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])

model.fit(X_train, y_train, epochs=200)

Epoch 1/200
1060/1060 [==============================] - 1s 789us/step - loss: 3981.8284 - mean_squared_error: 3981.8284
Epoch 2/200
1060/1060 [==============================] - 1s 773us/step - loss: 2893.8346 - mean_squared_error: 2893.8346
Epoch 3/200
1060/1060 [==============================] - 1s 775us/step - loss: 2518.0960 - mean_squared_error: 2518.0960
Epoch 4/200
1060/1060 [==============================] - 1s 761us/step - loss: 1943.1096 - mean_squared_error: 1943.1096
Epoch 5/200
1060/1060 [==============================] - 1s 763us/step - loss: 1527.3771 - mean_squared_error: 1527.3771
Epoch 6/200
1060/1060 [==============================] - 1s 773us/step - loss: 984.6956 - mean_squared_error: 984.6956
Epoch 7/200
1060/1060 [==============================] - 1s 794us/step - loss: 591.8249 - mean_squared_error: 591.8249
Epoch 8/200
1060/1060 [==============================] - 1s 798us/step - loss: 341.5531 - mean_squared_error: 341.5531
Epoch 9/200
1060/1060 [===============

1060/1060 [==============================] - 1s 1ms/step - loss: 46.7935 - mean_squared_error: 46.7935
Epoch 141/200
1060/1060 [==============================] - 1s 1ms/step - loss: 45.2081 - mean_squared_error: 45.2081
Epoch 142/200
1060/1060 [==============================] - 1s 1ms/step - loss: 46.9385 - mean_squared_error: 46.9385
Epoch 143/200
1060/1060 [==============================] - 1s 1ms/step - loss: 43.7789 - mean_squared_error: 43.7789
Epoch 144/200
1060/1060 [==============================] - 1s 1ms/step - loss: 50.7025 - mean_squared_error: 50.7025
Epoch 145/200
1060/1060 [==============================] - 1s 1ms/step - loss: 39.3566 - mean_squared_error: 39.3566
Epoch 146/200
1060/1060 [==============================] - 1s 1ms/step - loss: 54.2789 - mean_squared_error: 54.2789
Epoch 147/200
1060/1060 [==============================] - 1s 1ms/step - loss: 40.4752 - mean_squared_error: 40.4752
Epoch 148/200
1060/1060 [==============================] - 1s 1ms/step - loss:

In [15]:
pred_train= model.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train)))

pred= model.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred))) 

69.47817878517914
67.70385363256187


In [16]:
y_pred = model.predict(X_test)

mae=mean_absolute_error(y_test, y_pred)
print('MAE: %.4f' % mae)

mse = mean_squared_error(y_test, y_pred)
print('MSE: %.3f' % mse)
print('RMSE: %.3f' % (mse*(1/2.0)))

y_pred = model.predict(X_test)
r2 = r2_score(y_test,y_pred)
print('R-Squared: %.4f' % r2)

MAE: 43.0059
MSE: 4583.812
RMSE: 2291.906
R-Squared: -0.6170


In [17]:
#SQL database update
code='nn2'
process='Neural_network'
datab='standard'
depdelay='no'

cur.execute("INSERT OR REPLACE INTO parametres(codi,proces,DepDelay, base_dades, MAE, R2, MSE) VALUES(?,?,?,?,?,?,?)",(code,process,depdelay,datab,'%.3f' % mae,'%.3f' % r2,'%.3f' % mse))
conn.commit()




##### Check the SQL database

In [18]:
print(pd.read_sql_query("SELECT * from parametres", conn))

         codi          proces DepDelay    base_dades     MAE     R2       MSE
0         ls1           Lasso      yes      standard   3.548  0.987    44.190
1         ls2           Lasso       no      standard   3.548  0.987    44.190
2   objective  Neural_network       no  objective_dt  35.110  0.129  2828.555
3         dt1   Decision_Tree      yes      standard   5.913  0.976    79.421
4         dt2   Decision_Tree      yes      standard   9.374  0.936   198.610
5         dt3   Decision_Tree       no      standard  14.571  0.831   517.535
6         dt4   Decision_Tree      yes  standard/min   9.374  0.831   198.610
7         dt5   Decision_Tree      yes   no standard   9.818  0.831   227.035
8         dt6   Decision_Tree       no   no standard  14.717  0.831   493.721
9         nn1  Neural_network      yes      standard  42.966 -0.615  4578.544
10        nn2  Neural_network       no      standard  43.006 -0.617  4583.812
